In [1]:
import pandas as pd
import re
from tqdm import tqdm

In [2]:
import os
os.getcwd()

'/Users/ryleyun/Desktop/Github/Ryles_Practice/Projects/Final'

In [3]:
# 한국어 비출판물 말뭉치로 경로 변경
os.chdir('../../DATA/korean_nonpup')

In [ ]:
# 현재 디렉토리 내의 파일을 확인한 후 sjml 파일만 파일리스트에 넣기

files = []

for i in os.listdir():
    if '.sjml' in i:
        files.append(i)

In [ ]:
# 파일 불러와서 문장 집어 넣기

sentences = []

for file in tqdm(files):
    data = open(file, 'r')
    text = data.readlines()
    
    for line in text:
        if '<p>' in line:
            new_line = re.sub("\.\s|\?\s|!\s", "\n", line)
            temp = new_line.split('\n')
            for pre_line in temp:
                if len(re.sub("<.?p>", "", pre_line).strip()) >= 10 and len(re.sub("<.?p>", "", pre_line).strip()) <= 200:
                    sentences.append(re.sub("<.?p>", "", pre_line).strip())

In [ ]:
pre_sentences = []

for sent in tqdm(sentences):
    if sent not in pre_sentences:
        pre_sentences.append(re.sub('^[\'\"]|[\'\"]$', "", sent))

In [10]:
df = pd.DataFrame(pre_sentences)
df.to_csv("korean_nonpup_lines.csv", encoding='utf-8')

In [5]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer
from collections import defaultdict
from konlpy.tag import Mecab
import pandas as pd
import numpy as np
import re

#파일 불러온 후 전처리
#lineterminater를 붙힌 이유
#ParserError: Error tokenizing data. 
#C error: Buffer overflow caught-possible malformed input file.라는 에러가 나기 때문입니다

love_poem = pd.read_csv('korean_nonpup_lines.csv',lineterminator='\n') 
Love_Poem = love_poem['0'].values.tolist()
clean_lovepoem = []

In [6]:
for j in Love_Poem:
	cleanpoem = re.sub('[…().!?,""---―\ufeff\u200bknsjQuery1319700039250="65">jQuery1319700039250="68">\r\xa0~‘’;/]','',str(j))
	edited_poem = re.sub('[\u2e80-\u2eff\u31c0-\u31ef\u3200-\u32ff\u3400-\u4dbf\u4e00-\u9fbf\uf900-\ufaff]', '', cleanpoem)
	clean_lovepoem.append(edited_poem)
    
#문단 나누기
data_lovepoem = []
for i in clean_lovepoem:
	splited = i.split('\n')
	data_lovepoem.append(splited)
splited_lovepoem = []
for j in data_lovepoem:
	for k in j:
		if k:
			splited_lovepoem.append(k)
            
#핵심키워드 추출
mecab = Mecab() 
preprocessed_docs = []
for i in splited_lovepoem:
    token_list = [] 
    for token in mecab.pos(i):
        if token[1] in ['NNG']:
            token_list.append(token[0])
    preprocessed_docs.append(" ".join(token_list))
vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(preprocessed_docs)
word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx
    
#30%이상 100% 미만인 키워드 추출
keywords_lovelists = []
for i, sent in enumerate(preprocessed_docs):
    for token in sent.split():
        matrix = sp_matrix[i, word2id[token]]
        if 0.3 <= matrix < 0.95:
            keywords_lovelists.append(token)
            
#중복된 키워드 제거
key_lovepoem = [] 
for v in keywords_lovelists:
	if v not in key_lovepoem and len(v)>1:
        	key_lovepoem.append(v)
print(key_lovepoem)

/var/folders/s5/j2ttz_bd7d500bsb_zg9212w0000gn/T/ipykernel_4515/643097175.py:2: FutureWarning: Possible set difference at position 10
  cleanpoem = re.sub('[…().!?,""---―\ufeff\u200bknsjQuery1319700039250="65">jQuery1319700039250="68">\r\xa0~‘’;/]','',str(j))


['딸아', '올해', '작년', '이맘때', '해동', '빈자리', '실감', '동안', '고생', '시련', '고통', '감내', '요즘', '한편', '미안', '아버지', '거대', '그늘', '아래', '가족', '행복', '마음', '여유', '대사', '결혼식', '엄마', '최선', '한계', '민서', '귀뜸', '가정', '안주', '역할', '환경', '사람', '친정', '풍습', '가풍', '가법', '역지사지', '고사성어', '주장', '사고', '행운', '화목', '기원', '공포', '장르', '관습', '대물림', '해석', '대부분', '영화', '설정', '결말', '예상', '애스터', '전작', '오컬트', '유발', '장애아', '조장', '등장', '공동체', '윤리', '해이', '도구', '기능', '이해', '그림', '마을', '전달', '사자', '남성', '제물', '존재', '처단', '일시', '권위', '마무리', '의식', '이후', '상상', '주인공', '설명', '방식', '대상', '심연', '엔딩', '비판', '시간', '표현', '유학', '날짜', '오빠', '가차', '생활', '정리', '출발', '나중', '성인', '그때', '연락', '교생', '실습', '시작', '신청', '사범', '성격', '학생', '선생', '부담', '수업', '의문', '술자리', '고민', '하루', '일요일', '종일', '준비', '정장', '평소', '대학교', '후배', '슬리퍼', '신발', '첫날', '신고', '불편', '사이즈', '물집', '생각', '교직', '실내화', '말씀', '밴드', '양말', '호의', '태도', '처음', '자괴감', '한참', '부모', '인생', '고난', '역경', '성신', '여중', '초심', '내일', '다짐', '수요일', '용기', '덕분', '이름', '자신', '기억', '해사', '교무실', '연세', '남자', '이야기', '

In [ ]:
import random
import os, glob
import os.path
import import_ipynb
import love_poem_crawler, oldpoem_crawling

lovepoem_datalist = love_poem_crawler.key_lovepoem

#중복된 키워드 제거
cleaned_poemdata = []
for v in lovepoem_datalist:
    if v not in cleaned_poemdata:
        cleaned_poemdata.append(v)
        
#시 데이터와 사진 데이터와 맵핑
foldersdir = r"/home/ubuntu/anaconda3/envs/Minseok/Portfolio/TeamProject/AutoCrawler-master/download"
folders = os.listdir(foldersdir)
files_list = []
for i in folders:
	lists = foldersdir + '/' + i
	files = os.listdir(lists)
	files_list.append(files)
oldpoem_lists = dict(zip(folders, files_list))
randoms = random.choice(cleaned_poemdata)
for key, value in oldpoem_lists.items():
	if key == randoms:
		random_value = random.choice(value)
		print(key,':',random_value)